In [1]:
txt_best_grids = "best_results_cup_nn.txt"
grid_results_name = 'grid_results_cup_nn_v5.csv'

In [2]:
# !pip install ipython-autotime
# !pip install -U scikit-learn
%load_ext autotime

time: 0 ns (started: 2021-01-22 13:47:18 +01:00)


In [3]:
!python --version

Python 3.8.6
time: 32 ms (started: 2021-01-22 13:47:18 +01:00)


In [4]:
import os

os.chdir('../../')
os.getcwd()

'C:\\Users\\Luca\\Desktop\\Elle\\University\\0 - UNIPI\\ML - Machine Learning\\Workspace\\ml-project'

time: 0 ns (started: 2021-01-22 13:47:18 +01:00)


In [5]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

from src.utility import *

time: 2.81 s (started: 2021-01-22 13:47:18 +01:00)


In [6]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

time: 31 ms (started: 2021-01-22 13:47:21 +01:00)


# Import Dataset

In [7]:
def load_cup():
    ml_cup = np.delete(np.genfromtxt('cup/ML-CUP20-TR.csv', 
                                     delimiter=','), obj=0, axis=1)
    return ml_cup[:, :-2], ml_cup[:, -2:]

def load_cup_blind():
    return np.delete(np.genfromtxt('cup/ML-CUP20-TS.csv',
                                   delimiter=','), obj=0, axis=1)
    
def mean_euclidean_error(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    return np.mean(np.linalg.norm(y_pred - y_true, axis=1))

time: 0 ns (started: 2021-01-22 13:47:21 +01:00)


In [8]:
X, y = load_cup()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

time: 15 ms (started: 2021-01-22 13:47:21 +01:00)


In [9]:
print(X.shape)
print(y.shape)

(1524, 10)
(1524, 2)
time: 0 ns (started: 2021-01-22 13:47:21 +01:00)


In [10]:
# data = np.genfromtxt('ML-CUP20-TR.csv', delimiter=',', dtype=np.float64)
# X = data[:, 1:-2]
# y = data[:, -2:]

# print(X.shape)
# print(y.shape)

time: 0 ns (started: 2021-01-22 13:47:21 +01:00)


In [11]:
# Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.20, random_state=42)
# print(Xtrain.shape)
# print(ytrain.shape)
# print(Xval.shape)
# print(yval.shape)

time: 0 ns (started: 2021-01-22 13:47:21 +01:00)


# Grid Search - Gradient Descent

In [12]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import L2

# Function to create model, required for KerasRegressor
def create_model(learn_rate=0.01, momentum=0, neurons_layer_1=10,
                 init_mode='uniform', activation='relu',  weight_decay = 0.0,
                 nesterov = False, neurons_layer_2 = 5):
  # create model
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(10)))
    model.add(Dense(neurons_layer_1, activation=activation,
                    kernel_initializer=init_mode, kernel_regularizer= L2(weight_decay)))
    model.add(Dense(neurons_layer_2, activation=activation,
                    kernel_initializer=init_mode, kernel_regularizer= L2(weight_decay)))
    model.add(Dense(2))
    optimizer = SGD(
      learning_rate=learn_rate, momentum=momentum, nesterov=nesterov)
    # Compile model
    model.compile(optimizer=optimizer,
              loss=euclidean_distance_loss)
    return model

BATCH_SIZE = len(X_train)

# create model
model = KerasRegressor(build_fn=create_model, verbose=2) #verbose = 0
# define the grid search parameters
batch_size = [32, 64, 128]
epochs = [1000]
learn_rate = [0.0001, 0.001, 0.01]
momentum = [0.7, 0.8, 0.9, 1]
nesterov = [True]
neurons_layer_1 = [130, 70]
neurons_layer_2 = [30]
init_mode = ['glorot_uniform']
activation = ['tanh', 'sigmoid']
weight_decay = [0.001, 0.0001, 0.0003, 0.00001]

param_grid = dict(batch_size=batch_size, epochs=epochs, learn_rate=learn_rate,
                  momentum=momentum, neurons_layer_1=neurons_layer_1, 
                  init_mode=init_mode, activation=activation, weight_decay= weight_decay,
                  nesterov= nesterov, neurons_layer_2= neurons_layer_2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs= 6, cv=3, return_train_score=True, verbose=True)
grid_result = grid.fit(X_train, y_train)


Fitting 3 folds for each of 576 candidates, totalling 1728 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  4.3min
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed: 19.1min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed: 36.7min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed: 51.8min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed: 87.8min
[Parallel(n_jobs=6)]: Done 1728 out of 1728 | elapsed: 111.8min finished


Epoch 1/1000
10/10 - 0s - loss: 55.5032
Epoch 2/1000
10/10 - 0s - loss: 47.6550
Epoch 3/1000
10/10 - 0s - loss: 33.6889
Epoch 4/1000
10/10 - 0s - loss: 20.8323
Epoch 5/1000
10/10 - 0s - loss: 17.1462
Epoch 6/1000
10/10 - 0s - loss: 16.7826
Epoch 7/1000
10/10 - 0s - loss: 16.7077
Epoch 8/1000
10/10 - 0s - loss: 16.4515
Epoch 9/1000
10/10 - 0s - loss: 16.1736
Epoch 10/1000
10/10 - 0s - loss: 15.8017
Epoch 11/1000
10/10 - 0s - loss: 15.2268
Epoch 12/1000
10/10 - 0s - loss: 14.3166
Epoch 13/1000
10/10 - 0s - loss: 12.5250
Epoch 14/1000
10/10 - 0s - loss: 10.0901
Epoch 15/1000
10/10 - 0s - loss: 8.7965
Epoch 16/1000
10/10 - 0s - loss: 8.6007
Epoch 17/1000
10/10 - 0s - loss: 8.4627
Epoch 18/1000
10/10 - 0s - loss: 8.3135
Epoch 19/1000
10/10 - 0s - loss: 8.1779
Epoch 20/1000
10/10 - 0s - loss: 8.0490
Epoch 21/1000
10/10 - 0s - loss: 7.8941
Epoch 22/1000
10/10 - 0s - loss: 7.7071
Epoch 23/1000
10/10 - 0s - loss: 7.5119
Epoch 24/1000
10/10 - 0s - loss: 7.2941
Epoch 25/1000
10/10 - 0s - loss: 7.

Epoch 204/1000
10/10 - 0s - loss: 2.6958
Epoch 205/1000
10/10 - 0s - loss: 2.7001
Epoch 206/1000
10/10 - 0s - loss: 2.6883
Epoch 207/1000
10/10 - 0s - loss: 2.6969
Epoch 208/1000
10/10 - 0s - loss: 2.6887
Epoch 209/1000
10/10 - 0s - loss: 2.6921
Epoch 210/1000
10/10 - 0s - loss: 2.6859
Epoch 211/1000
10/10 - 0s - loss: 2.6852
Epoch 212/1000
10/10 - 0s - loss: 2.6806
Epoch 213/1000
10/10 - 0s - loss: 2.6830
Epoch 214/1000
10/10 - 0s - loss: 2.6828
Epoch 215/1000
10/10 - 0s - loss: 2.6858
Epoch 216/1000
10/10 - 0s - loss: 2.6766
Epoch 217/1000
10/10 - 0s - loss: 2.6801
Epoch 218/1000
10/10 - 0s - loss: 2.6810
Epoch 219/1000
10/10 - 0s - loss: 2.6841
Epoch 220/1000
10/10 - 0s - loss: 2.6769
Epoch 221/1000
10/10 - 0s - loss: 2.6849
Epoch 222/1000
10/10 - 0s - loss: 2.6850
Epoch 223/1000
10/10 - 0s - loss: 2.6731
Epoch 224/1000
10/10 - 0s - loss: 2.6738
Epoch 225/1000
10/10 - 0s - loss: 2.6679
Epoch 226/1000
10/10 - 0s - loss: 2.6662
Epoch 227/1000
10/10 - 0s - loss: 2.6609
Epoch 228/1000
1

Epoch 404/1000
10/10 - 0s - loss: 2.4988
Epoch 405/1000
10/10 - 0s - loss: 2.4987
Epoch 406/1000
10/10 - 0s - loss: 2.4933
Epoch 407/1000
10/10 - 0s - loss: 2.4966
Epoch 408/1000
10/10 - 0s - loss: 2.4910
Epoch 409/1000
10/10 - 0s - loss: 2.4884
Epoch 410/1000
10/10 - 0s - loss: 2.4993
Epoch 411/1000
10/10 - 0s - loss: 2.4989
Epoch 412/1000
10/10 - 0s - loss: 2.5017
Epoch 413/1000
10/10 - 0s - loss: 2.4977
Epoch 414/1000
10/10 - 0s - loss: 2.4942
Epoch 415/1000
10/10 - 0s - loss: 2.4910
Epoch 416/1000
10/10 - 0s - loss: 2.4814
Epoch 417/1000
10/10 - 0s - loss: 2.4888
Epoch 418/1000
10/10 - 0s - loss: 2.4899
Epoch 419/1000
10/10 - 0s - loss: 2.4840
Epoch 420/1000
10/10 - 0s - loss: 2.4888
Epoch 421/1000
10/10 - 0s - loss: 2.4865
Epoch 422/1000
10/10 - 0s - loss: 2.4867
Epoch 423/1000
10/10 - 0s - loss: 2.4816
Epoch 424/1000
10/10 - 0s - loss: 2.4761
Epoch 425/1000
10/10 - 0s - loss: 2.4772
Epoch 426/1000
10/10 - 0s - loss: 2.4819
Epoch 427/1000
10/10 - 0s - loss: 2.4792
Epoch 428/1000
1

Epoch 604/1000
10/10 - 0s - loss: 2.3598
Epoch 605/1000
10/10 - 0s - loss: 2.3661
Epoch 606/1000
10/10 - 0s - loss: 2.3656
Epoch 607/1000
10/10 - 0s - loss: 2.3605
Epoch 608/1000
10/10 - 0s - loss: 2.3668
Epoch 609/1000
10/10 - 0s - loss: 2.3632
Epoch 610/1000
10/10 - 0s - loss: 2.3587
Epoch 611/1000
10/10 - 0s - loss: 2.3598
Epoch 612/1000
10/10 - 0s - loss: 2.3606
Epoch 613/1000
10/10 - 0s - loss: 2.3626
Epoch 614/1000
10/10 - 0s - loss: 2.3589
Epoch 615/1000
10/10 - 0s - loss: 2.3602
Epoch 616/1000
10/10 - 0s - loss: 2.3518
Epoch 617/1000
10/10 - 0s - loss: 2.3699
Epoch 618/1000
10/10 - 0s - loss: 2.3696
Epoch 619/1000
10/10 - 0s - loss: 2.3521
Epoch 620/1000
10/10 - 0s - loss: 2.3606
Epoch 621/1000
10/10 - 0s - loss: 2.3687
Epoch 622/1000
10/10 - 0s - loss: 2.3522
Epoch 623/1000
10/10 - 0s - loss: 2.3524
Epoch 624/1000
10/10 - 0s - loss: 2.3494
Epoch 625/1000
10/10 - 0s - loss: 2.3502
Epoch 626/1000
10/10 - 0s - loss: 2.3593
Epoch 627/1000
10/10 - 0s - loss: 2.3542
Epoch 628/1000
1

Epoch 804/1000
10/10 - 0s - loss: 2.2672
Epoch 805/1000
10/10 - 0s - loss: 2.2659
Epoch 806/1000
10/10 - 0s - loss: 2.2668
Epoch 807/1000
10/10 - 0s - loss: 2.2729
Epoch 808/1000
10/10 - 0s - loss: 2.2687
Epoch 809/1000
10/10 - 0s - loss: 2.2658
Epoch 810/1000
10/10 - 0s - loss: 2.2639
Epoch 811/1000
10/10 - 0s - loss: 2.2661
Epoch 812/1000
10/10 - 0s - loss: 2.2659
Epoch 813/1000
10/10 - 0s - loss: 2.2613
Epoch 814/1000
10/10 - 0s - loss: 2.2616
Epoch 815/1000
10/10 - 0s - loss: 2.2655
Epoch 816/1000
10/10 - 0s - loss: 2.2614
Epoch 817/1000
10/10 - 0s - loss: 2.2685
Epoch 818/1000
10/10 - 0s - loss: 2.2693
Epoch 819/1000
10/10 - 0s - loss: 2.2697
Epoch 820/1000
10/10 - 0s - loss: 2.2624
Epoch 821/1000
10/10 - 0s - loss: 2.2687
Epoch 822/1000
10/10 - 0s - loss: 2.2635
Epoch 823/1000
10/10 - 0s - loss: 2.2579
Epoch 824/1000
10/10 - 0s - loss: 2.2621
Epoch 825/1000
10/10 - 0s - loss: 2.2641
Epoch 826/1000
10/10 - 0s - loss: 2.2655
Epoch 827/1000
10/10 - 0s - loss: 2.2652
Epoch 828/1000
1

In [13]:
import json

grid_results_name_str = f'Results wrt: {grid_results_name} \n'
grid_params_str = f'Grid on: {json.dumps(grid_result.param_grid)} \n'
grid_results_str = f'Best: {grid_result.best_score_} using {grid_result.best_params_} \n'

with open(txt_best_grids, "a") as file_object:
    file_object.write(grid_results_name_str)
    file_object.write(grid_params_str)
    file_object.write(grid_results_str)
    file_object.write('\n')

time: 0 ns (started: 2021-01-22 15:39:19 +01:00)


In [14]:
print(grid_results_str)

Best: -2.752460161844889 using {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 1000, 'init_mode': 'glorot_uniform', 'learn_rate': 0.01, 'momentum': 0.9, 'nesterov': True, 'neurons_layer_1': 130, 'neurons_layer_2': 30, 'weight_decay': 1e-05} 

time: 0 ns (started: 2021-01-22 15:39:19 +01:00)


In [15]:
import pandas as pd


df = pd.DataFrame(grid.cv_results_)[['rank_test_score','mean_test_score',
                                     'std_test_score','mean_train_score',
                                     'std_train_score','param_activation',
                                     'param_batch_size','param_epochs',
                                     'param_init_mode', 'param_learn_rate',
                                     'param_momentum','param_nesterov',
                                     'param_neurons_layer_1', 'param_neurons_layer_2',
                                     'param_weight_decay','mean_fit_time']].sort_values(by='rank_test_score')

df.rename(columns={'param_activation': 'activation',
                   'param_batch_size': 'batch_size',
                   'param_epochs': 'epochs',
                   'param_init_mode': 'init_mode',
                   'param_learn_rate': 'learn_rate',
                   'param_momentum': 'momentum',
                   'param_init_mode': 'init_mode',
                   'param_weight_decay': 'weight_decay',
                   'param_nesterov': 'nesterov',
                   'param_neurons_layer_1': 'neurons_layer_1',
                   'param_neurons_layer_2': 'neurons_layer_2',
                   'mean_test_score': 'mean_val_score',
                   'std_test_score': 'std_val_score',                
                   'rank_test_score': 'rank_val_score'}, inplace=True)

df.mean_train_score *= -1
df.mean_val_score *= -1
df

,rank_val_score,mean_val_score,std_val_score,mean_train_score,std_train_score,activation,batch_size,epochs,init_mode,learn_rate,momentum,nesterov,neurons_layer_1,neurons_layer_2,weight_decay,mean_fit_time
563,1,2.752460e+00,8.205573e-02,2.222884e+00,3.120425e-02,sigmoid,128,1000,glorot_uniform,0.01,0.9,True,130,30,1e-05,14.006078
459,2,2.756296e+00,6.246307e-02,2.255241e+00,6.139289e-02,sigmoid,64,1000,glorot_uniform,0.01,0.8,True,130,30,1e-05,21.586086
451,3,2.769325e+00,7.172792e-02,2.384264e+00,4.357651e-02,sigmoid,64,1000,glorot_uniform,0.01,0.7,True,130,30,1e-05,20.821568
555,4,2.781920e+00,6.660798e-02,2.458510e+00,4.253278e-02,sigmoid,128,1000,glorot_uniform,0.01,0.8,True,130,30,1e-05,14.136749
551,5,2.790404e+00,8.002081e-02,2.548099e+00,2.538287e-02,sigmoid,128,1000,glorot_uniform,0.01,0.7,True,70,30,1e-05,12.345481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,572,5.108635e+07,7.223224e+07,5.108635e+07,7.223224e+07,tanh,64,1000,glorot_uniform,0.01,1,True,70,30,0.001,18.421093
184,573,8.834129e+07,1.114119e+08,8.834131e+07,1.114119e+08,tanh,64,1000,glorot_uniform,0.01,1,True,130,30,0.001,18.284473
89,574,9.509918e+07,1.192974e+08,9.509919e+07,1.192975e+08,tanh,32,1000,glorot_uniform,0.01,1,True,130,30,0.0001,32.963306
93,575,1.649544e+08,2.270731e+08,1.649543e+08,2.270730e+08,tanh,32,1000,glorot_uniform,0.01,1,True,70,30,0.0001,31.592616


time: 47 ms (started: 2021-01-22 15:39:19 +01:00)


In [16]:
df.to_csv(grid_results_name, index= False)

time: 16 ms (started: 2021-01-22 15:39:19 +01:00)


In [17]:
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# import csv

# with open(single_grid_results_filename, mode='w') as grid:
#     grid = csv.writer(grid, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     for mean, stdev, param in zip(means, stds, params):
#       grid.writerow([mean, stdev, param])

time: 0 ns (started: 2021-01-22 15:39:19 +01:00)


In [18]:
# from sklearn.externals import joblib

# # #save your model or results
# # joblib.dump(grid_result, 'gs_object_monk3.pkl')
# # #load your model for further usage
# # boh = joblib.load("gs_object_monk3.pkl")

# # joblib.dump(grid.best_estimator_, 'gs_best_estimator_monk3.pkl', compress = 1)
# # filename = 'finalized_model.sav'
# # pickle.dump(model, open(filename, 'wb'))

time: 0 ns (started: 2021-01-22 15:39:19 +01:00)
